<a href="https://www.kaggle.com/code/fatemamoharam/spaceship-titanic?scriptVersionId=127735763" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [5]:
! pip install lazypredict keras-tuner -q

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import multiprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# from lazypredict.Supervised import LazyClassifier

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.callbacks import ReduceLROnPlateau
import keras_tuner

## Setup tensorboard

In [8]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [9]:
# Clear any logs from previous runs
!rm -rf /tmp/tb_logs
!mkdir /tmp/tb_logs

In [10]:
# ! kill $(ps -e | grep 'tensorboard' | awk '{print $1}')
%tensorboard --logdir /tmp/tb_logs

Reusing TensorBoard on port 6006 (pid 84), started 0:06:17 ago. (Use '!kill 84' to kill it.)

# Read and explore the data

In [11]:
ROOT = '/kaggle/input/spaceship-titanic'
TRAIN = ROOT + '/' + 'train.csv'
TEST = ROOT + '/' + 'test.csv'

In [12]:
train_df = pd.read_csv(TRAIN)
test_df  = pd.read_csv(TEST)
train_df.head()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

## Processing numerical columns

In [13]:
numCols = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

Normalization

In [14]:
train_mean = {}
train_std = {}

for col in numCols:
    train_mean[col] = train_df[col].mean()
    train_std[col]  = train_df[col].std()

    train_df[col] = ( train_df[col] - train_df[col].mean() ) / train_df[col].std()
    test_df[col]  = ( test_df[col]  - train_df[col].mean() ) / train_df[col].std()

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

## Processing Categorical Features

In [15]:
catCols = ['HomePlanet','Destination','Cabin','Name']

In [16]:
for col in catCols:
    print(f"{col} : \n {train_df[col].unique()}\n\n")

HomePlanet : 
 ['Europa' 'Earth' 'Mars' nan]


Destination : 
 ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]


Cabin : 
 ['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']


Name : 
 ['Maham Ofracculy' 'Juanna Vines' 'Altark Susent' ... 'Fayey Connon'
 'Celeon Hontichre' 'Propsh Hontichre']




### Breaking 'Cabin' down

In [17]:
max_train = max(train_df.count())
max_test = max(test_df.count())

In [18]:
side = np.zeros((max_train),dtype=np.float32)
side_tst = np.zeros((max_test),dtype=np.float32)
for i, cabin in enumerate(train_df['Cabin'].values):
    if cabin:
        side[i] = 1 if str(cabin).split('/')[-1] == 'P' else 0
    else:
        side[i] = np.NaN
    
for i, cabin in enumerate(test_df['Cabin'].values):
    if cabin:
        side_tst[i] = 1 if str(cabin).split('/')[-1] == 'P' else 0
    else:
        side_tst[i] = np.NaN

In [19]:
num_cabin = np.zeros_like(side,dtype=np.float32)
num_cabin_tst = np.zeros_like(side_tst,dtype=np.float32)
for i, cabin in enumerate(train_df['Cabin'].values):
    try:
        num_cabin[i] = float(str(cabin).split('/')[1])
    except(IndexError):
        num_cabin[i] = np.NaN
    
for i, cabin in enumerate(test_df['Cabin'].values):
    try:
        num_cabin_tst[i] = float(str(cabin).split('/')[1])
    except(IndexError):
        num_cabin_tst[i] = np.NaN

In [20]:
deck = np.zeros_like(side,dtype=str)
deck_tst = np.zeros_like(side_tst,dtype=str)

for i, cabin in enumerate(train_df['Cabin'].values):
    try:
        deck[i] = str(cabin).split('/')[0]
    except(IndexError):
        deck[i] = 'NaN'
    
for i, cabin in enumerate(test_df['Cabin'].values):
    try:
        deck_tst[i] = str(cabin).split('/')[0]
    except(IndexError):
        deck_tst[i] = 'NaN'

In [16]:
side, num_cabin, deck

(array([1., 0., 0., ..., 0., 0., 0.], dtype=float32),
 array([   0.,    0.,    0., ..., 1500.,  608.,  608.], dtype=float32),
 array(['B', 'F', 'A', ..., 'G', 'E', 'E'], dtype='<U1'))

In [17]:
side_tst, num_cabin_tst, deck_tst

(array([0., 0., 0., ..., 1., 1., 0.], dtype=float32),
 array([   3.,    4.,    0., ...,  296.,  297., 1498.], dtype=float32),
 array(['G', 'F', 'C', ..., 'D', 'D', 'G'], dtype='<U1'))

In [21]:
col = 'Cabin Number'
num_cabin = pd.Series(num_cabin,name=col)
train_mean[col] = num_cabin.mean()
train_std[col] = num_cabin.std()

train_df[col] = ( num_cabin - num_cabin.mean() ) / num_cabin.std()
test_df[col] = ( num_cabin_tst - num_cabin.mean() ) / num_cabin.std()

In [19]:
train_df[col].describe()

count    8.494000e+03
mean    -1.437136e-08
std      1.000000e+00
min     -1.172897e+00
25%     -8.461524e-01
50%     -3.386966e-01
75%      7.787807e-01
max      2.527281e+00
Name: Cabin Number, dtype: float64

In [20]:
test_df[col].describe()

count    4177.000000
mean        0.019167
std         1.006058
min        -1.172897
25%        -0.832965
50%        -0.309392
75%         0.833482
max         2.519467
Name: Cabin Number, dtype: float64

## One Hot Encoding

In [22]:
deck_df = pd.get_dummies(data=deck,prefix='Deck_',dummy_na=True)
home_df = pd.get_dummies(data=train_df['HomePlanet'],prefix='Home_',dummy_na=True)
dest_df = pd.get_dummies(data=train_df['Destination'],prefix='Dest_',dummy_na=True)
side_df = pd.get_dummies(data=side,prefix='Side_',dummy_na=True)



train_df = pd.concat([train_df.drop(['Cabin','HomePlanet','Destination'],axis=1),deck_df,home_df,dest_df],axis=1)

In [23]:
deck_df = pd.get_dummies(data=deck_tst,prefix='Deck_',dummy_na=True)
home_df = pd.get_dummies(data=test_df['HomePlanet'],prefix='Home_',dummy_na=True)
dest_df = pd.get_dummies(data=test_df['Destination'],prefix='Dest_',dummy_na=True)
side_df = pd.get_dummies(data=side_tst,prefix='Side_',dummy_na=True)


test_df = pd.concat([test_df.drop(['Cabin','HomePlanet','Destination'],axis=1),deck_df,home_df,dest_df],axis=1)

In [23]:
test_df.columns

Index(['PassengerId', 'CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Cabin Number', 'Deck__A',
       'Deck__B', 'Deck__C', 'Deck__D', 'Deck__E', 'Deck__F', 'Deck__G',
       'Deck__T', 'Deck__n', 'Deck__nan', 'Home__Earth', 'Home__Europa',
       'Home__Mars', 'Home__nan', 'Dest__55 Cancri e', 'Dest__PSO J318.5-22',
       'Dest__TRAPPIST-1e', 'Dest__nan'],
      dtype='object')

In [24]:
train_df.columns

Index(['PassengerId', 'CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'Transported', 'Cabin Number',
       'Deck__A', 'Deck__B', 'Deck__C', 'Deck__D', 'Deck__E', 'Deck__F',
       'Deck__G', 'Deck__T', 'Deck__n', 'Deck__nan', 'Home__Earth',
       'Home__Europa', 'Home__Mars', 'Home__nan', 'Dest__55 Cancri e',
       'Dest__PSO J318.5-22', 'Dest__TRAPPIST-1e', 'Dest__nan'],
      dtype='object')

## Bool features

In [24]:
boolCols = ['CryoSleep','VIP','Transported']
for col in boolCols:
    train_df[col] = 1 * train_df[col].values 
    if col != 'Transported':
        test_df[col] = 1 * test_df[col].values

# Cleanup

In [25]:
train_df = train_df.drop(['Name','PassengerId'],axis=1)
test_id = test_df['PassengerId']
test_df = test_df.drop(['Name','PassengerId'],axis=1)

In [27]:
pd.Series(side)

0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8688    1.0
8689    0.0
8690    0.0
8691    0.0
8692    0.0
Length: 8693, dtype: float32

In [26]:
train_df['Side'] = side
test_df['Side'] = side_tst

In [29]:
train_df.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin Number,...,Deck__nan,Home__Earth,Home__Europa,Home__Mars,Home__nan,Dest__55 Cancri e,Dest__PSO J318.5-22,Dest__TRAPPIST-1e,Dest__nan,Side
0,0,0.702054,0,-0.337006,-0.284257,-0.287300,-0.273720,-0.266082,0,-1.172897,...,0,0,1,0,0,0,0,1,0,1.0
1,0,-0.333213,0,-0.173518,-0.278672,-0.245957,0.209255,-0.227678,1,-1.172897,...,0,1,0,0,0,0,0,1,0,0.0
2,0,2.013391,1,-0.272511,1.934808,-0.287300,5.633703,-0.223314,0,-1.172897,...,0,0,1,0,0,0,0,1,0,0.0
3,0,0.287947,0,-0.337006,0.511901,0.326231,2.654919,-0.097629,0,-1.172897,...,0,0,1,0,0,0,0,1,0,0.0
4,0,-0.885355,0,0.117460,-0.240819,-0.037588,0.223331,-0.264336,1,-1.170944,...,0,1,0,0,0,0,0,1,0,0.0


In [30]:
test_df.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin Number,Deck__A,...,Deck__nan,Home__Earth,Home__Europa,Home__Mars,Home__nan,Dest__55 Cancri e,Dest__PSO J318.5-22,Dest__TRAPPIST-1e,Dest__nan,Side
0,1,27.0,0,-2.170361e-17,-2.922326e-18,5.024462e-18,-1.836891e-17,-1.294934e-17,-1.167036,0,...,0,1,0,0,0,0,0,1,0,0.0
1,0,19.0,0,-2.170361e-17,9.000000e+00,5.024462e-18,2.823000e+03,-1.294934e-17,-1.165083,0,...,0,1,0,0,0,0,0,1,0,0.0
2,1,31.0,0,-2.170361e-17,-2.922326e-18,5.024462e-18,-1.836891e-17,-1.294934e-17,-1.172897,0,...,0,0,1,0,0,1,0,0,0,0.0
3,0,38.0,0,-2.170361e-17,6.652000e+03,5.024462e-18,1.810000e+02,5.850000e+02,-1.170944,0,...,0,0,1,0,0,0,0,1,0,0.0
4,0,20.0,0,1.000000e+01,-2.922326e-18,6.350000e+02,-1.836891e-17,-1.294934e-17,-1.163129,0,...,0,1,0,0,0,0,0,1,0,0.0


In [31]:
test_df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin Number,Deck__A,Deck__B,Deck__C,...,Deck__nan,Home__Earth,Home__Europa,Home__Mars,Home__nan,Dest__55 Cancri e,Dest__PSO J318.5-22,Dest__TRAPPIST-1e,Dest__nan,Side
count,4.186000e+03,4.195000e+03,4.171000e+03,4.179000e+03,4.176000e+03,4.197000e+03,4177.000000,4277.000000,4277.000000,4277.000000,...,4277.0,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000
mean,2.865815e+01,2.192663e+02,4.394843e+02,1.772955e+02,3.030524e+02,3.107100e+02,0.019167,0.022913,0.084639,0.083002,...,0.0,0.529109,0.234276,0.216273,0.020341,0.196633,0.090718,0.691139,0.021510,0.487257
std,1.417907e+01,6.070113e+02,1.527663e+03,5.608211e+02,1.117186e+03,1.246995e+03,1.006058,0.149644,0.278376,0.275918,...,0.0,0.499210,0.423595,0.411751,0.141182,0.397499,0.287241,0.462078,0.145095,0.499896
min,-6.593009e-17,-2.170361e-17,-2.922326e-18,5.024462e-18,-1.836891e-17,-1.294934e-17,-1.172897,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.900000e+01,-2.170361e-17,-2.922326e-18,5.024462e-18,-1.836891e-17,-1.294934e-17,-0.832965,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.600000e+01,-2.170361e-17,-2.922326e-18,5.024462e-18,-1.836891e-17,-1.294934e-17,-0.309392,0.000000,0.000000,0.000000,...,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,3.700000e+01,5.300000e+01,7.800000e+01,3.300000e+01,5.000000e+01,3.600000e+01,0.833482,0.000000,0.000000,0.000000,...,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,7.900000e+01,1.156700e+04,2.527300e+04,8.292000e+03,1.984400e+04,2.227200e+04,2.519467,1.000000,1.000000,1.000000,...,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Balancing data

In [27]:
train_df['Transported'].value_counts()

1    4378
0    4315
Name: Transported, dtype: int64

In [28]:
train_df = train_df.groupby('Transported')
train_df = train_df.apply(lambda x: x.sample(train_df.size().min()).reset_index(drop=True))

In [29]:
train_df['Transported'].value_counts()

0    4315
1    4315
Name: Transported, dtype: int64

In [30]:
y_train = train_df['Transported']
train_df = train_df.drop('Transported',axis=1)

In [31]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

## Data Splitting

In [32]:
 x_train, x_val, y_train, y_val = train_test_split(train_df.values, y_train.values, test_size = 0.15, random_state =55, stratify = y_train)

In [33]:
 x_train.shape, x_val.shape, y_train.shape, y_val.shape

((7335, 28), (1295, 28), (7335,), (1295,))

In [89]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train[:,np.newaxis]))
val_ds = tf.data.Dataset.from_tensor_slices((x_val,y_val[:,np.newaxis]))

In [90]:
for i in train_ds.take(1).as_numpy_iterator():
    print(i)
    print(i[0].shape,i[1].shape)

(array([ 0.        ,  0.70205359,  0.        ,  0.18045477, -0.28425707,
        1.11505669, -0.27371977, -0.16483544,  2.09552836,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ]), array([0]))
(28,) (1,)


# Classifiers

In [ ]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(x_train, x_val, y_train, y_val)
models

In [ ]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=150)
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
y_pred = clf.predict(test_df.values)
sub = pd.DataFrame.from_dict({
    "PassengerId": test_id,
    "Transported": y_pred,
})
sub['Transported'] = sub['Transported'] == 1

sub.head()
sub.to_csv("/kaggle/working/submission.csv",index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=40)
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier([16,8],max_iter=2000)
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

clf = MLPClassifier([32,16],max_iter=2000)
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
clf = MLPClassifier([128,64],max_iter=2000)
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
clf = MLPClassifier([8,4,2],max_iter=2000)
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

clf = QuadraticDiscriminantAnalysis()
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

In [ ]:
from sklearn.ensemble import BaggingClassifier

clf = BaggingClassifier(SVC())
clf.fit(x_train,y_train)
print(classification_report(y_val,clf.predict(x_val)))

# Deeplerning approach

In [34]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [35]:
LEARNING_RATE = 0.01
PATIENCE = 100
EPOCHS = 1000
CHK = '/kaggle/working/checkpoints/chk.tf'

In [36]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=1e-8)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10, mode='min')

def configure_checkpoints(path):
    return keras.callbacks.ModelCheckpoint(
        filepath=path,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)

checkpoint = configure_checkpoints('/kaggle/working/checkpoints/chk.tf')
tb = keras.callbacks.TensorBoard("/tmp/tb_logs")
callbacks = [reduce_lr, early_stopping, checkpoint,tb]

In [ ]:
def train_and_plot(model,checkpoint_path,x_train,y_train,x_val,y_val,epochs=EPOCHS,batch_size=512):
    checkpoint = configure_checkpoints(checkpoint_path)
    history=model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_val,y_val),
        callbacks=[lr_scheduler,checkpoint,early_stopping]
    )

    num=len(history.history['loss'])

    plt.plot(range(num),history.history['loss'],label = 'training loss')
    plt.plot(range(num),history.history['val_loss'],label = 'validation loss')

    plt.legend()

    plt.xlabel("Epochs ")
    plt.ylabel('Loss')

    plt.figure()

    plt.plot(range(num),history.history['accuracy'],label = 'training accuracy')
    plt.plot(range(num),history.history['val_accuracy'],label = 'validation accuracy')

    plt.legend()
    plt.xlabel("Epochs ")
    return history

In [68]:
model = keras.Sequential(
[
#     layers.Dropout(0.3,),
    layers.Dense(128,activation='tanh',input_shape=x_train.shape[1:]+(1,),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(32,activation='tanh'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1,activation='sigmoid',name='output')
], name='one-layer'
)
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss = keras.losses.BinaryCrossentropy()
model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

print(model.summary())

SyntaxError: invalid syntax (4224328031.py, line 11)

In [ ]:
history = train_and_plot(
    model,
    CHK,
    x_train,
    y_train,
    x_val,
    y_val,
    epochs=EPOCHS
)

model.load_weights(CHK)
model.evaluate(x_val,y_val)

In [ ]:
y_pred = model.predict(test_df.values)
sub = pd.DataFrame.from_dict({
    "PassengerId": test_id,
    "Transported": y_pred.ravel(),
})
sub['Transported'] = sub['Transported'] == 1

sub.head()
sub.to_csv("/kaggle/working/submission.csv",index=False)

In [80]:
train_ds.take(5)

<TakeDataset shapes: ((28,), ()), types: (tf.float64, tf.int64)>

HPTuning

In [37]:
def build_model(hp):
    dense = hp.Int('dense',min_value=1,max_value=16,sampling='log')
    dropout = hp.Float('dropout',min_value=0.0,max_value=0.5,step=0.1)
    activation = hp.Choice('activation',['tanh','relu'])
    num_layers = hp.Int('num_layers',min_value=1,max_value=16,sampling='log')
    lr = hp.Choice('lr',[1e-5,1e-4,1e-3,1e-2,1e-1,1.0])
    l=[]
    for i in range(0,num_layers):
        if i == 0:
            l.append(layers.Dense(dense,activation=activation,input_shape=train_df.values.shape[1:]))
        else :
            l.append(layers.Dense(dense,activation=activation))
        l.append(layers.BatchNormalization())
        l.append(layers.Dropout(dropout))
    l.append(layers.Dense(1,activation='sigmoid',name='output'))
        
    model = keras.Sequential(l, name='hp-tune')
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    loss = keras.losses.BinaryCrossentropy()
    model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])
    return model

In [38]:
tuner = keras_tuner.RandomSearch(
    build_model,
    max_trials=100,
    objective="val_accuracy",
    # Set a directory to store the intermediate results.
    directory="/tmp/tb",
)

In [ ]:
tuner.search(
    x_train,
    y_train,
    validation_data=(x_val,y_val),
    epochs=100,
    # Use the TensorBoard callback.
    # The logs will be write to "/tmp/tb_logs".
    callbacks=callbacks,
)

Trial 19 Complete [00h 01m 29s]
val_accuracy: 0.500386118888855

Best val_accuracy So Far: 0.8007721900939941
Total elapsed time: 00h 15m 35s

Search: Running Trial #20

Value             |Best Value So Far |Hyperparameter
9                 |8                 |dense
0.1               |0.1               |dropout
tanh              |relu              |activation
6                 |1                 |num_layers
1e-05             |0.01              |lr

Epoch 1/100
230/230 [==============================] - 4s 7ms/step - loss: 0.7700 - accuracy: 0.5902 - val_loss: 0.6969 - val_accuracy: 0.6185
Epoch 2/100
230/230 [==============================] - 1s 5ms/step - loss: 0.7683 - accuracy: 0.5952 - val_loss: 0.7032 - val_accuracy: 0.6224
Epoch 3/100
230/230 [==============================] - 1s 5ms/step - loss: 0.7411 - accuracy: 0.6026 - val_loss: 0.6885 - val_accuracy: 0.6324
Epoch 4/100
230/230 [==============================] - 1s 5ms/step - loss: 0.7417 - accuracy: 0.6037 - val_loss: 0.677

In [ ]:
! tensorboard dev upload --logdir /tmp/tb_logs \
    --name "spaceship titanic tuning - random search" \